Chapter 5 stuff

From Chapter 5 

Deutsch algorithm

In [ ]:
import numpy as np
import random
import cirq
from cirq import H, X, CNOT, measure

q0, q1 = cirq.LineQubit.range(2)
secret_function = [random.randint(0,1) for _ in range(2)]

def make_oracle(a, b, c):
    if c[0]:
        yield [CNOT(a,b), X(b)]
    if c[1]:
        yield CNOT(a,b)

def make_deutsch_circuit(d,e,f):
    c = cirq.Circuit()
    c.append([H(e), H(e), H(d)])
    c.append(f)
    c.append([H(d), measure(d, key='result')])
    return c

oracle = make_oracle(q0, q1, secret_function)
circuit = make_deutsch_circuit(q0, q1, oracle)
print(circuit)

simulator = cirq.Simulator()
result = simulator.run(circuit)
print(result)

Deutsch-Josza

In [ ]:

'''
Code adopted from: https://qiskit.org/textbook/ch-algorithms/deutsch-jozsa.html#4.-Qiskit-Implementation-
'''
# initialization
import numpy as np

# importing Qiskit
from qiskit import BasicAer
#from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, execute

# import basic plot tools
from qiskit.visualization import plot_histogram

np.random.seed(42)

# set the length of the n-bit input string. 
n = 3

# set the length of the n-bit input string. 
n = 3

%matplotlib inline

#Constant oracle
const_oracle = QuantumCircuit(n+1)

output = np.random.randint(2)
if output == 1:
    const_oracle.x(n)

#const_oracle.draw(output='mpl')
const_oracle.draw()


#Balanced oracle
balanced_oracle = QuantumCircuit(n+1)
b_str = "101"

balanced_oracle = QuantumCircuit(n+1)
b_str = "101"

# Place X-gates
for qubit in range(len(b_str)):
    if b_str[qubit] == '1':
        balanced_oracle.x(qubit)
balanced_oracle.draw()



balanced_oracle = QuantumCircuit(n+1)
b_str = "101"

# Place X-gates
for qubit in range(len(b_str)):
    if b_str[qubit] == '1':
        balanced_oracle.x(qubit)

# Use barrier as divider
balanced_oracle.barrier()

# Controlled-NOT gates
for qubit in range(n):
    balanced_oracle.cx(qubit, n)

balanced_oracle.barrier()
balanced_oracle.draw()


balanced_oracle = QuantumCircuit(n+1)
b_str = "101"

# Place X-gates
for qubit in range(len(b_str)):
    if b_str[qubit] == '1':
        balanced_oracle.x(qubit)

# Use barrier as divider
balanced_oracle.barrier()

# Controlled-NOT gates
for qubit in range(n):
    balanced_oracle.cx(qubit, n)

balanced_oracle.barrier()

# Place X-gates
for qubit in range(len(b_str)):
    if b_str[qubit] == '1':
        balanced_oracle.x(qubit)

# Show oracle
balanced_oracle.draw()


#implementation of the algo
dj_circuit = QuantumCircuit(n+1, n)

# Apply H-gates
for qubit in range(n):
    dj_circuit.h(qubit)

# Put qubit in state |->
dj_circuit.x(n)
dj_circuit.h(n)
dj_circuit.draw()

dj_circuit = QuantumCircuit(n+1, n)

# Apply H-gates
for qubit in range(n):
    dj_circuit.h(qubit)

# Put qubit in state |->
dj_circuit.x(n)
dj_circuit.h(n)

# Add oracle
dj_circuit += balanced_oracle
dj_circuit.draw()

dj_circuit = QuantumCircuit(n+1, n)

# Apply H-gates
for qubit in range(n):
    dj_circuit.h(qubit)

# Put qubit in state |->
dj_circuit.x(n)
dj_circuit.h(n)

# Add oracle
dj_circuit += balanced_oracle

# Repeat H-gates
for qubit in range(n):
    dj_circuit.h(qubit)
dj_circuit.barrier()

# Measure
for i in range(n):
    dj_circuit.measure(i, i)

# Display circuit
dj_circuit.draw()


# use local simulator
backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(dj_circuit, backend=backend, shots=shots).result()
answer = results.get_counts()

plot_histogram(answer)

Bernstein-vazirani Algorithm

In [ ]:
'''
This code is adopted from: https://github.com/qiskit-community/qiskit-community-tutorials/blob/master/Coding_With_Qiskit/ep6_Bernstein-Vazirani_Algorithm.ipynb
'''

#!pip install qiskit
import numpy as np
from qiskit import *
from qiskit.visualization import plot_histogram

np.random.seed(42)

s = input("Enter the secret bit string:\n")

n = len(s)

circuit = QuantumCircuit(n+1,n)

# Step 0

circuit.x(n) 
circuit.barrier() 

# Step 1

circuit.h(range(n+1)) 

circuit.barrier() 

# Step 2

for ii, yesno in enumerate(reversed(s)):
    if yesno == '1': 
        circuit.cx(ii, n)
    
circuit.barrier()

# Step 3

circuit.h(range(n+1)) 

circuit.barrier() 

circuit.measure(range(n), range(n))

%matplotlib inline
#circuit.draw(output='mpl')
circuit.draw()

#Running the algorithm
simulator = Aer.get_backend('qasm_simulator')
result = execute(circuit, backend=simulator, shots=1024).result()
plot_histogram(result.get_counts(circuit))

N-qubit QFT using Qiskit

In [ ]:
from qiskit.circuit.library import QFT

n = input(" Enter the number of qubits: \n")

qft_circuit = QFT(num_qubits = n)

qft_circuit.draw()

Quantum Phase estimation using cirq

In [ ]:

"""Creates and simulates a phase estimator circuit.
=== EXAMPLE OUTPUT ===
Testing with 8 qubits.
target=0.0000, estimate=0.0000=0/256
target=0.1000, estimate=0.1016=26/256
target=0.2000, estimate=0.1992=51/256
target=0.3000, estimate=0.3008=77/256
target=0.4000, estimate=0.3984=102/256
target=0.5000, estimate=0.5000=128/256
target=0.6000, estimate=0.6016=154/256
target=0.7000, estimate=0.6992=179/256
target=0.8000, estimate=0.8008=205/256
target=0.9000, estimate=0.8984=230/256
RMS Error: 0.0011
"""

'''
This code is adopted from: https://github.com/quantumlib/Cirq/blob/master/examples/phase_estimator.py
'''

import numpy as np
import cirq


def run_estimate(unknown_gate, qnum, repetitions):
    

    ancilla = cirq.LineQubit(-1)
    qubits = cirq.LineQubit.range(qnum)

    oracle_raised_to_power = [
        unknown_gate.on(ancilla).controlled_by(qubits[i])**(2**i)
        for i in range(qnum)
    ]
    circuit = cirq.Circuit(cirq.H.on_each(*qubits), oracle_raised_to_power,
                           cirq.QFT(*qubits, without_reverse=True)**-1,
                           cirq.measure(*qubits, key='phase'))

    return cirq.sample(circuit, repetitions=repetitions)


def experiment(qnum, repetitions=100):
    

    def example_gate(phi):
        
        gate = cirq.MatrixGate(
            matrix=np.array([[np.exp(2 * np.pi * 1.0j * phi), 0], [0, 1]]))
        return gate

    print(f'Testing with {qnum} qubits.')
    errors = []
    for target in np.arange(0, 1, 0.1):
        result = run_estimate(example_gate(target), qnum, repetitions)
        mode = result.data['phase'].mode()[0]
        guess = mode / 2**qnum
        print(f'target={target:0.4f}, '
              f'estimate={guess:0.4f}={mode}/{2**qnum}')
        errors.append((target - guess)**2)
    rms = np.sqrt(sum(errors) / len(errors))
    print(f'RMS Error: {rms:0.4f}\n')


def main(qnums = (2, 4, 8), repetitions=100):
    for qnum in qnums:
        experiment(qnum, repetitions=repetitions)


if __name__ == '__main__':
    main()

Simons algorithm using qiskit

In [ ]:
#!pip install 'qiskit==0.19'
#!pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src

'''
This is taken from: https://qiskit.org/textbook/ch-algorithms/simon.html
'''
# importing Qiskit
from qiskit import BasicAer
from qiskit.providers.ibmq import least_busy
from qiskit import QuantumCircuit, execute

# import basic plot tools
from qiskit.visualization import plot_histogram
from qiskit_textbook.tools import simon_oracle

import matplotlib.pyplot as plt

b = '110'

n = len(b)
simon_circuit = QuantumCircuit(n*2, n)

# Apply Hadamard gates before querying the oracle
simon_circuit.h(range(n))    
    
# Apply barrier for visual separation
simon_circuit.barrier()

simon_circuit += simon_oracle(b)

# Apply barrier for visual separation
simon_circuit.barrier()

# Apply Hadamard gates to the input register
simon_circuit.h(range(n))

# Measure qubits
simon_circuit.measure(range(n), range(n))
simon_circuit.draw('mpl')
plt.show()

# use local simulator
backend = BasicAer.get_backend('qasm_simulator')
shots = 1024
results = execute(simon_circuit, backend=backend, shots=shots).result()
counts = results.get_counts()
plot_histogram(counts)

# Calculate the dot product of the results
def bdotz(b, z):
    accum = 0
    for i in range(len(b)):
        accum += int(b[i]) * int(z[i])
    return (accum % 2)

print('b = ' + b)
for z in counts:
    print( '{}.{} = {} (mod 2) ({:.1f}%)'.format(b, z, bdotz(b,z), counts[z]*100/shots))


Shor algorithm

In [ ]:
#!pip install 'qiskit==0.19'

#!pip install 'qiskit==0.19'
#!pip install git+https://github.com/qiskit-community/qiskit-textbook.git#subdirectory=qiskit-textbook-src
#!pip install sympy

from qiskit import BasicAer
from qiskit.aqua import QuantumInstance
from qiskit.aqua.algorithms import Shor

number = Shor(N=15, a=7)
simulator = BasicAer.get_backend('qasm_simulator')
results_dictionary = number.run(QuantumInstance(backend=simulator, shots=5))
#result = results_dictionary['number']
print(results_dictionary)

Grover's algorithm 

In [ ]:
#!pip install 'qiskit==0.19'

'''
Grover's two-qubit search algorithm.
'''
from qiskit import QuantumRegister, ClassicalRegister, QuantumCircuit, execute, BasicAer
from qiskit.visualization import plot_histogram
import matplotlib.pyplot as plt

c = ClassicalRegister(2, 'c')
q = QuantumRegister(2,'q')
qc = QuantumCircuit(q,c)

qc.h([q[0]])
qc.h([q[1]])

qc.x([q[0]])
qc.x([q[1]])

qc.cz(0,1)

qc.x([q[0]])
qc.x([q[1]])

qc.h([q[0]])
qc.h([q[1]])

qc.z(q[0])
qc.z(q[1])

qc.cz(0,1)

qc.h([q[0]])
qc.h([q[1]])

qc.measure(q[0], c[0])
qc.measure(q[1], c[1])
qc.draw('mpl')
plt.show()

simulator = BasicAer.get_backend('qasm_simulator')
job = execute(qc, simulator, shots=1024)
result = job.result()
count = result.get_counts(qc)
plot_histogram(count)
plt.show()